# Angle Ply Laminate

---



In [ ]:
from dfply import *
import grama as gr
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Full DataFrame view
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from grama.models import make_composite_plate_tension
model = make_composite_plate_tension([-np.pi/4, +np.pi/4])

In [ ]:
model.printpretty()

This model has a large number of random variables, but no deterministic variables.

Let's generate sinew data to begin exploring this model.

In [ ]:
df_sweeps = model >> gr.ev_sinews(n_sweeps=10, df_det="nom")
df_sweeps.describe()

Observations:

- Among the limit states (outputs)
  + The longitudinal (11) `tension` states are largely non-critical (positive)
  + The transverse (22) `tension` states and `shear` states are frequently critical (negative)
  
Note however that a sinew design is not a Monte Carlo; the design above is not drawn from the distribution.
  
We can tell a lot more by visualizing these data.

In [ ]:
df_melted = \
    df_sweeps >> \
    gather("input", "x", model.var_rand) >> \
    gather("output", "y", model.outputs) >> \
    filter_by(X.sweep_var == X.input)

sns.relplot(
    data=df_melted,
    x="x",
    y="y",
    hue="sweep_ind",
    col="input",
    row="output",
    kind="line",
    facet_kws=dict(sharex=False, sharey=False)
)
plt.savefig("./sweeps_full.png")

This full sinew plot is a bit overwhelming; I save a local copy for zooming. Observations:

- A number of the limit states are fairly nonlinear; it is difficult to tell which from the zoomed-out version.

Sinew plots can be useful for debugging a model during development; the following plot focuses on the limit states and critical stresses for a single ply. If the model is implemented correctly the effects should be *diagonal*, in the sense that each critical stress should only have an affect on its associated limit state. Furthermore, each effect should be linear with slope one.

In [ ]:
df_lam0 = \
    df_melted >> \
    filter_by(
        (X.input == "sigma_11_t_0") |
        (X.input == "sigma_22_t_0") |
        (X.input == "sigma_11_c_0") |
        (X.input == "sigma_22_c_0") |
        (X.input == "sigma_12_s_0"),
        (X.output == "g_11_tension_0") |
        (X.output == "g_22_tension_0") |
        (X.output == "g_11_compression_0") |
        (X.output == "g_22_compression_0") |
        (X.output == "g_12_shear_0")
    )

sns.relplot(
    data=df_lam0,
    x="x",
    y="y",
    hue="sweep_ind",
    col="input",
    row="output",
    kind="line",
    facet_kws=dict(sharex=False, sharey=False)
)

Here we can see the diagonal relationship, as expected.

From inspecting the full sinew plot, it appears the shear modulii `g_12_shear_i` have a strange cross-effect:

In [ ]:
df_shear = \
    df_melted >> \
    filter_by(
        (X.input == "G12_0") |
        (X.input == "G12_1"),
        (X.output == "g_12_shear_0") |
        (X.output == "g_12_shear_1")
    )

sns.relplot(
    data=df_shear,
    x="x",
    y="y",
    hue="sweep_ind",
    col="input",
    row="output",
    kind="line",
    facet_kws=dict(sharex=False, sharey=False)
)

Observations:

- The shear modulus `g_12_shear_i` **decreases** the associated limit state `g_12_shear_i`
- The shear modulus `g_12_shear_i` **increases** the cross-associated limit state `g_12_shear_j`

Stiffening one ply causes that ply to take more of the total load, which reduces the load on the other ply. This explains the opposite effects observed above.

---

From close inspection of the full sinew plot, it appears the effect of certain variables switch across different sinews:

In [ ]:
df_shear = \
    df_melted >> \
    filter_by(
        (X.input == "G12_0") |
        (X.input == "G12_1"),
        (X.output == "g_11_tension_0") |
        (X.output == "g_11_tension_1")
    )

sns.relplot(
    data=df_shear,
    x="x",
    y="y",
    hue="sweep_ind",
    col="input",
    row="output",
    kind="line",
    facet_kws=dict(sharex=False, sharey=False)
)

Here we can see the shear modulii `G12_i` have an effect on the opposite ply `tension` that can change in sign depending on the values of the other random variables. Thus the shear modulii have a **non-monotone** effect on the longitudinal tension limit state.